# Template 

- Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)

In [1]:
%load_ext watermark

In [7]:
import pandas as pd
import pythran

In [9]:
# Run this cell before close.
%watermark
%watermark --iversion
%watermark -b -r -g

2020-06-26T01:19:21+00:00

CPython 3.7.7
IPython 7.15.0

compiler   : GCC 8.3.0
system     : Linux
release    : 4.19.76-linuxkit
machine    : x86_64
processor  : 
CPU cores  : 16
interpreter: 64bit
pythran 0.9.5
pandas  1.0.5

Git hash: 3eec8b191e966c58878e00a1a29f129a0c0ffcb1
Git repo: https://github.com/ysraell/examples.git
Git branch: master


In [4]:
def pi_approximate(n):
    step = 1.0 / n
    result = 0   
    for i in range(n):
     x = (i + 0.5) * step
     result += 4.0 / (1.0 + x * x)
    return step * result


In [6]:
%timeit pi_approximate(1000000)

94.1 ms ± 747 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%load_ext pythran.magic

In [11]:
>>> %%pythran
>>> #pythran export pi_approximate_pythran(int)
>>> import numpy as np
>>> def pi_approximate_pythran(n):
...     step = 1.0 / n
...     result = 0   
...     for i in range(n):
...         x = (i + 0.5) * step
...         result += 4.0 / (1.0 + x * x)
...     return step * result

In [12]:
%timeit pi_approximate_pythran(1000000)

1.19 ms ± 2.46 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
>>> %%pythran -fopenmp
>>> #pythran export pi_approximate_pythran_omp(int)
>>> import numpy as np
>>> def pi_approximate_pythran_omp(n):
...     step = 1.0 / n
...     result = 0
...     #omp parallel for reduction(+:result)
...     for i in range(n):
...         x = (i + 0.5) * step
...         result += 4.0 / (1.0 + x * x)
...     return step * result

In [14]:
%timeit pi_approximate_pythran_omp(1000000)

196 µs ± 23.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
>>> import numpy as np
>>> def pi_numpy_style(n):
...     step = 1.0 / n
...     x = (np.arange(0, n, dtype=np.float64) + 0.5) * step
...     return step * np.sum(4. / (1. + x ** 2))

In [16]:
%timeit pi_numpy_style(1000000)

7.22 ms ± 490 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
>>> %%pythran
>>> #pythran export pi_numpy_style_pythran(int)
>>> import numpy as np
>>> def pi_numpy_style_pythran(n):
...     step = 1.0 / n
...     x = (np.arange(0, n, dtype=np.float64) + 0.5) * step
...     return step * np.sum(4. / (1. + x ** 2))

In [18]:
%timeit pi_numpy_style_pythran(1000000)

1.49 ms ± 14.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [19]:
>>> %%pythran -DUSE_BOOST_SIMD -march=native
>>> #pythran export pi_numpy_style_pythran_simd(int)
>>> import numpy as np
>>> def pi_numpy_style_pythran_simd(n):
...     step = 1.0 / n
...     x = (np.arange(0, n, dtype=np.float64) + 0.5) * step
...     return step * np.sum(4. / (1. + x ** 2))

In [20]:
%timeit pi_numpy_style_pythran_simd(1000000)

511 µs ± 979 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# pairwise

In [22]:
>>> def pairwise_distance(X):
...     return np.sqrt(((X[:, None, :] - X) ** 2).sum(-1))

In [24]:
>>> size = 100
>>> args32 = np.random.random((size, size)).astype(np.float32)
>>> args64 = np.random.random((size, size)).astype(np.float64)  #cast useless
>>> %timeit pairwise_distance(args32)
>>> %timeit pairwise_distance(args64)

3.14 ms ± 84.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.85 ms ± 53.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
>>> %%pythran -fopenmp
>>> import numpy as np
>>> #pythran export pairwise_distance_pythran(float32[][])
>>> #pythran export pairwise_distance_pythran(float64[:,:])
>>> def pairwise_distance_pythran(X):
...     print(X.dtype)
...     return np.sqrt(((X[:, None, :] - X) ** 2).sum(-1))

In [30]:
>>> print(pairwise_distance_pythran(args32).dtype)
>>> %timeit pairwise_distance_pythran(args32)
>>> print(pairwise_distance_pythran(args64).dtype)
>>> %timeit pairwise_distance_pythran(args64)

float32
769 µs ± 2.55 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
float64
887 µs ± 2.99 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [32]:
args32.shape

(100, 100)

In [44]:
matrix = np.random.random((1000, 5)).astype(np.float32)

In [45]:
%timeit pairwise_distance(matrix)

27.1 ms ± 438 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [50]:
%timeit pairwise_distance_pythran(matrix)

16.4 ms ± 739 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [43]:
a[3].shape

(10,)